In [ ]:
# Installing packages

!pip install langchain openai transformers tiktoken faiss-cpu

# LangChain Prompt

```markdown
BasePromptTemplate --> PipelinePromptTemplate
                       StringPromptTemplate --> PromptTemplate
                                                FewShotPromptTemplate
                                                FewShotPromptWithTemplates
                       BaseChatPromptTemplate --> AutoGPTPrompt
                                                  ChatPromptTemplate --> AgentScratchPadChatPromptTemplate



BaseMessagePromptTemplate --> MessagesPlaceholder
                              BaseStringMessagePromptTemplate --> ChatMessagePromptTemplate
                                                                  HumanMessagePromptTemplate
                                                                  AIMessagePromptTemplate
                                                                  SystemMessagePromptTemplate

PromptValue --> StringPromptValue
                ChatPromptValue

```

## PromptTemplate


In [8]:
import langchain
from langchain.prompts.prompt import PromptTemplate

In [ ]:
my_template = PromptTemplate(
    input_variables= ['cuisine'],
    template="i want to open a {cuisine} restaurant. sugget me a good name"
)

print(my_template)

input_variables=['cuisine'] template='i want to open a {cuisine} restaurant. sugget me a good name'


In [ ]:
PromptTemplate.format(my_template, cuisine="india")

'i want to open a india restaurant. sugget me a good name'

In [ ]:
my_template_2 = PromptTemplate(
    input_variables= ['cuisine', 'location'],
    template= "i want to open a {cuisine} restaurant in {location}. suggest me a good name"
)
print(PromptTemplate.format(my_template_2, cuisine="Mexican", location="Vizag"))

i want to open a Mexican restaurant in Vizag. suggest me a good name


## FewShotPromptTemplate

In [7]:
# class langchain.prompts.few_shot.FewShotPromptTemplate

from langchain.prompts.few_shot import FewShotPromptTemplate

In [ ]:
examples = [
    {   "question": "When was the founder of craigslist born?",
        "answer": "December 6, 1952"
     },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": "Joseph Ball"
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": "No"
    }
]

qa_template = PromptTemplate(
    input_variables= ["question", "answer"],
    template= "Question: {question}\nAnswer: {answer}"
)


few_shot_template = FewShotPromptTemplate(
    input_variables= ["question"],
    example_prompt = qa_template,
    prefix = "please answer the following question based on these examples:\n\nQuestion: {question}",
    suffix = "\n\nAnswer: ",
    examples= examples
)

my_prompt = few_shot_template.format(question="who is the prime minister of india?")

print(my_prompt)

please answer the following question based on these examples:

Question: who is the prime minister of india?

Question: When was the founder of craigslist born?
Answer: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Answer: Joseph Ball

Question: Are both the directors of Jaws and Casino Royale from the same country?
Answer: No



Answer: 


In [3]:
import openai
from langchain.llms import OpenAI
from secret_key import openapi_key, serp_key, huggingface_key
import os
os.environ['OPENAI_API_KEY'] = openapi_key
os.environ['HUGGINGFACEHUB_API_TOKEN'] = huggingface_key

In [ ]:
llm = OpenAI(verbose=True)
name = llm(my_prompt)
print(name)

 Narendra Modi


## FewShotPromptWithTemplates

I couldnt find much difference between the **FewShotPromptTemplates** and FewShotPromptWithTemplates if you found it just let me know or fork the branch and create a pull request i will merge it. Thank you.

## Example Selector

### Semantic Similarity Example Selector

In [6]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector, MaxMarginalRelevanceExampleSelector
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.llms import OpenAI

In [17]:
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
    {"input": "boy", "output": "king"},
    {"input": "girl", "output": "queen"},
    {"input": "school", "output": "student"},
    {"input": "police station", "output": "police man"}
]

example_prompt = PromptTemplate(
    input_variables=['input','output'],
    template = "input:{input}\noutput:{output}"
)

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    FAISS,
    k=3
)

similar_prompts = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt= example_prompt,
    prefix = "give the anotomy of every input",
    suffix = "Input: {user_input}\nOutput:",
    input_variables = ['user_input']

)

mychain = LLMChain(llm=OpenAI(),prompt=similar_prompts,verbose=True)
print(mychain.predict(user_input='fire'))



> Entering new LLMChain chain...
Prompt after formatting:
give the anotomy of every input

input:police station
output:police man

input:school
output:student

input:windy
output:calm

Input: fire
Output:

> Finished chain.
 Smoke


### maximal marginal relevance (MMR)

In [16]:
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
    {"input": "boy", "output": "king"},
    {"input": "girl", "output": "queen"},
    {"input": "school", "output": "student"},
    {"input": "police station", "output": "police man"}
]

example_prompt = PromptTemplate(
    input_variables=['input','output'],
    template = "input:{input}\noutput:{output}"
)

example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    FAISS,
    k=3
)

similar_prompts = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt= example_prompt,
    prefix = "give the anotomy of every input",
    suffix = "Input: {user_input}\nOutput:",
    input_variables = ['user_input']

)

mychain = LLMChain(llm=OpenAI(),prompt=similar_prompts,verbose=True)
print(mychain.predict(user_input='fire'))



> Entering new LLMChain chain...
Prompt after formatting:
give the anotomy of every input

input:police station
output:police man

input:energetic
output:lethargic

input:girl
output:queen

Input: fire
Output:

> Finished chain.
 Smoke


### n-gram overlap

I dont want to cover this since its typical n-gram approach

### length

his example selector selects which examples to use based on length. This is useful when you are worried about constructing a prompt that will go over the length of the context window. For longer inputs, it will select fewer examples to include, while for shorter inputs it will select more.

## OutputParser

### StructuredOutputParser

In [4]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [5]:
Ress = [ResponseSchema(name="bad string", description="This is a poorly formated user input string"),
        ResponseSchema(name="good string", description="This is your reponse, a reformatted response")]

output = StructuredOutputParser.from_response_schemas(Ress)
format_instruction = output.get_format_instructions()
print(format_instruction)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad string": string  // This is a poorly formated user input string
	"good string": string  // This is your reponse, a reformatted response
}
```


In [9]:
template = """
you might be given a poorly formated string from a user.

Reformat it and make sure all the words are spelled correctly including country, state and city names.

{format_instruction}

User Input: {user_input}

your response:

"""
prompt = PromptTemplate(
    input_variables=['user_input'],
    partial_variables={"format_instruction": format_instruction},
    template = template
)

mychain = LLMChain(llm=OpenAI(),prompt=prompt,verbose=True)
print(mychain.predict(user_input="thr shd be some user accptance test that we need to do"))



> Entering new LLMChain chain...
Prompt after formatting:

you might be given a poorly formated string from a user.

Reformat it and make sure all the words are spelled correctly including country, state and city names.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad string": string  // This is a poorly formated user input string
	"good string": string  // This is your reponse, a reformatted response
}
```

User Input: thr shd be some user accptance test that we need to do

your response: 



> Finished chain.

```json
{
	"bad string": "thr shd be some user accptance test that we need to do",
	"good string": "There should be some user acceptance tests that we need to do"
}
```


## AutoGPT

In [ ]:
# Under Construction......!!!!!!!!!!!!


# LangChain Memory

## Chat Messages

In [10]:
from langchain.memory import ChatMessageHistory

In [15]:
history = ChatMessageHistory()

history.add_user_message("hi")

history.add_user_message("what is your name")

history.add_ai_message("i am good thanks for asking")

In [16]:
history

ChatMessageHistory(messages=[HumanMessage(content='hi'), HumanMessage(content='what is your name'), AIMessage(content='i am good thanks for asking')])

## MemoryTypes

### Conversation Buffer

In [18]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [19]:
mychain = ConversationChain(
    memory = ConversationBufferMemory(),
    verbose = True,
    llm = OpenAI()
)

mychain.predict(input="hi")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hi
AI:

> Finished chain.


" Hi there! It's nice to meet you. How can I help you today?"

In [20]:
mychain.predict(input = "my name is sekhar, i want to know who is prime minister of india")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: my name is sekhar, i want to know who is prime minister of india
AI:

> Finished chain.


' The current Prime Minister of India is Narendra Modi. He has been in office since 2014. Is there anything else I can help you with today?'

In [21]:
mychain.predict(input="what is my name")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: my name is sekhar, i want to know who is prime minister of india
AI:  The current Prime Minister of India is Narendra Modi. He has been in office since 2014. Is there anything else I can help you with today?
Human: what is my name
AI:

> Finished chain.


'  You said your name is Sekhar. Is that correct?'

In [25]:
mychain.memory.load_memory_variables({})

{'history': "Human: hi\nAI:  Hi there! It's nice to meet you. How can I help you today?\nHuman: my name is sekhar, i want to know who is prime minister of india\nAI:  The current Prime Minister of India is Narendra Modi. He has been in office since 2014. Is there anything else I can help you with today?\nHuman: what is my name\nAI:   You said your name is Sekhar. Is that correct?"}

### Conversation Buffer Window

In [26]:
from langchain.memory import ConversationBufferWindowMemory

In [28]:
_my_chain = ConversationChain(
    memory = ConversationBufferWindowMemory(k=2),
    llm = OpenAI(),
    verbose = True
)

_my_chain.predict(input = "hi")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hi
AI:

> Finished chain.


" Hi there! It's nice to meet you. How can I help you today?"

In [29]:
_my_chain.predict(input = "my name is sekhar")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: my name is sekhar
AI:

> Finished chain.


' Nice to meet you Sekhar! What can I do for you today?'

In [30]:
_my_chain.predict(input = "who is the pm of india")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: my name is sekhar
AI:  Nice to meet you Sekhar! What can I do for you today?
Human: who is the pm of india
AI:

> Finished chain.


' The Prime Minister of India is Narendra Modi. He was sworn in as the Prime Minister on May 30, 2014.'

In [31]:
_my_chain.predict(input = "what is the time now")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: my name is sekhar
AI:  Nice to meet you Sekhar! What can I do for you today?
Human: who is the pm of india
AI:  The Prime Minister of India is Narendra Modi. He was sworn in as the Prime Minister on May 30, 2014.
Human: what is the time now
AI:

> Finished chain.


' The current time is 8:43 PM.'

In [32]:
_my_chain.predict(input = "what is my name")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: who is the pm of india
AI:  The Prime Minister of India is Narendra Modi. He was sworn in as the Prime Minister on May 30, 2014.
Human: what is the time now
AI:  The current time is 8:43 PM.
Human: what is my name
AI:

> Finished chain.


" I don't know what your name is."

### Conversation Summary

In [33]:
from langchain.llms import OpenAI
from langchain.memory import ConversationSummaryMemory
llm = OpenAI(temperature=0)
conversation_with_summary = ConversationChain(
    llm=llm,
    memory=ConversationSummaryMemory(llm=OpenAI()),
    verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


" Hi there! I'm doing great. I'm currently helping a customer with a technical issue. How about you?"

In [34]:
conversation_with_summary.predict(input="i am just learning how you are working thats it.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human asked the AI what it was doing, and the AI responded that it was helping a customer with a technical issue.
Human: i am just learning how you are working thats it.
AI:

> Finished chain.


" I'm helping a customer with a technical issue. I'm troubleshooting the issue and providing them with the best solution. I'm also providing them with resources and guidance to help them resolve the issue on their own."

In [36]:
conversation_with_summary.memory.buffer

'\nThe human asked the AI what it was doing, and the AI responded that it was helping a customer with a technical issue. It was troubleshooting the issue and providing them with the best solution, as well as resources and guidance to help the customer resolve the issue on their own.'

### Conversation Summary Buffer

In [37]:
from langchain.memory import ConversationSummaryBufferMemory

In [38]:
from langchain.chains import ConversationChain

conversation_with_summary = ConversationChain(
    llm=llm,
    memory=ConversationSummaryBufferMemory(llm=OpenAI(), max_token_limit=40),
    verbose=True,
)
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


" Hi there! I'm doing great. I'm currently helping a customer with a technical issue. How about you?"

### Backed by a Vector Store

In [41]:
from datetime import datetime
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
import faiss
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS


embedding_size = 1536 # Dimensions of the OpenAIEmbeddings
index = faiss.IndexFlatL2(embedding_size)
embedding_fn = OpenAIEmbeddings().embed_query
vectorstore = FAISS(embedding_fn, index, InMemoryDocstore({}), {})


In [42]:
# In actual usage, you would set `k` to be a higher value, but i use k=1 to show that
# the vector lookup still returns the semantically relevant information
retriever = vectorstore.as_retriever(search_kwargs=dict(k=1))
memory = VectorStoreRetrieverMemory(retriever=retriever)

# When added to an agent, the memory object can save pertinent information from conversations or used tools
memory.save_context({"input": "My favorite food is pizza"}, {"output": "that's good to know"})
memory.save_context({"input": "My favorite sport is soccer"}, {"output": "..."})
memory.save_context({"input": "I don't the Celtics"}, {"output": "ok"}) #

In [43]:
# Notice the first result returned is the memory pertaining to tax help, which the language model deems more semantically relevant
# to a 1099 than the other documents, despite them both containing numbers.
print(memory.load_memory_variables({"prompt": "what sport should i watch?"})["history"])

input: My favorite sport is soccer
output: ...


In [46]:
retriever = vectorstore.as_retriever(search_kwargs=dict(k=1))
memory = VectorStoreRetrieverMemory(retriever=retriever)

llm = OpenAI(temperature=0)
_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
{history}

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=_DEFAULT_TEMPLATE
)
conversation_with_summary = ConversationChain(
    llm=llm,
    prompt=PROMPT,
    memory=memory,
    verbose=True
)
conversation_with_summary.predict(input="Hi, my name is sekhar, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: Hi, my name is sekhar, what's up?
response:  Hi Sekhar, nice to meet you. I'm doing well. How about you?

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: Hi, my name is sekhar, what's up?
AI:

> Finished chain.


" Hi Sekhar, nice to meet you. I'm doing well. How about you?"

In [47]:
conversation_with_summary.predict(input="what's my favorite sport?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: My favorite sport is soccer
output: ...

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: what's my favorite sport?
AI:

> Finished chain.


' You told me earlier that your favorite sport is soccer.'

# Chains

Chains encode a sequence of calls to components like models, document retrievers, other Chains, etc.

In [ ]:
from langchain.chains import LLMChain
llm = OpenAI()
name = llm("I want to open a restaurant for indian food. suggest me a fancy name for this")
print(name)



Spice Bazaar


In [ ]:
my_template = PromptTemplate(
    input_variables=['cuisine', 'location'],
    template="I want to open a restaurant for {cuisine} food at {location}. suggest me a fancy name for it"
)

my_chain = LLMChain(llm=llm,prompt=my_template, verbose=True)
print(my_chain.predict(cuisine="india", location="Vizag"))



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for india food at Vizag. suggest me a fancy name for it

> Finished chain.


Mumbai Masala  


In [ ]:
name_prompt = PromptTemplate(
    input_variables=["cuisine"],
    template="I want to open a restaurant for {cuisine} food. suggest me a fancy name for it."
)

menu_prompt = PromptTemplate(
    input_variables=['restaurant_name'],
    template = "suggest me some menu items for {restaurant_name}."
)

name_chain = LLMChain(llm = OpenAI(), prompt = name_prompt, verbose=True)
menu_chain = LLMChain(llm = OpenAI(), prompt = menu_prompt, verbose = True)


In [ ]:
from langchain.chains import SimpleSequentialChain

s_chain = SimpleSequentialChain(chains=[name_chain, menu_chain], verbose = True)
print(s_chain("india"))



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for india food. suggest me a fancy name for it.

> Finished chain.


Maharaja's Palace Cuisine


> Entering new LLMChain chain...
Prompt after formatting:
suggest me some menu items for 

Maharaja's Palace Cuisine.

> Finished chain.


-Tandoori Chicken
-Butter Chicken
-Lamb Rogan Josh
-Chicken Tikka Masala
-Saag Paneer
-Aloo Gobi
-Matar Paneer
-Korma
-Palak Paneer
-Kadai Paneer
-Naan Bread
-Raita
-Mango Lassi
-Gulab Jamun

> Finished chain.
{'input': 'india', 'output': '\n\n-Tandoori Chicken\n-Butter Chicken\n-Lamb Rogan Josh\n-Chicken Tikka Masala\n-Saag Paneer\n-Aloo Gobi\n-Matar Paneer\n-Korma\n-Palak Paneer\n-Kadai Paneer\n-Naan Bread\n-Raita\n-Mango Lassi\n-Gulab Jamun'}


In [ ]:
from langchain.chains import SequentialChain

name_prompt = PromptTemplate(
    input_variables=["cuisine", "location"],
    template="I want to open a restaurant for {cuisine} food at {location}. suggest me a fancy name for it."
)

menu_prompt = PromptTemplate(
    input_variables=['restaurant_name'],
    template = "suggest me some menu items for {restaurant_name}."
)

name_chain = LLMChain(llm = OpenAI(), prompt = name_prompt, verbose=True, output_key="restaurant_name")
menu_chain = LLMChain(llm = OpenAI(), prompt = menu_prompt, verbose = True, output_key="menu_items")

my_restaurant = SequentialChain(chains=[name_chain, menu_chain],
                                input_variables=['cuisine', 'location'],
                                output_variables=['restaurant_name', 'menu_items'],
                                verbose=True)


print(my_restaurant({"cuisine":"indian", "location":"Vizag"}))



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for indian food at Vizag. suggest me a fancy name for it.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
suggest me some menu items for 

Maharajah's Tandoori Delight.

> Finished chain.

> Finished chain.
{'cuisine': 'indian', 'location': 'Vizag', 'restaurant_name': "\n\nMaharajah's Tandoori Delight", 'menu_items': '\n\n-Tandoori Chicken\n-Naan\n-Chicken Tikka Masala\n-Lamb Rogan Josh\n-Butter Chicken\n-Vegetable Korma\n-Spicy Potato Curry\n-Chana Masala\n-Biryani\n-Saag Paneer\n-Mango Lassi\n-Gulab Jamun'}


In [ ]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="bigscience/bloom-1b7",
    task="text-generation"
)

In [ ]:
prompt = PromptTemplate(
    input_variables=["city"],
    template="Describe a perfect day in {city}?",
)
llmchain = LLMChain(llm=llm, prompt=prompt)

llmchain.run("Paris")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


' A day in Paris is a day that you will never forget'

In [ ]:
name_prompt = PromptTemplate(
    input_variables=["cuisine", "location"],
    template="I want to open a restaurant for {cuisine} food at {location}. suggest me a fancy name for it."
)

menu_prompt = PromptTemplate(
    input_variables=['restaurant_name'],
    template = "suggest me some menu items for {restaurant_name}."
)

name_chain = LLMChain(llm = llm, prompt = name_prompt, verbose=True, output_key="restaurant_name")
menu_chain = LLMChain(llm = llm, prompt = menu_prompt, verbose = True, output_key="menu_items")

my_restaurant = SequentialChain(chains=[name_chain, menu_chain],
                                input_variables=['cuisine', 'location'],
                                output_variables=['restaurant_name', 'menu_items'],
                                verbose=True)


print(my_restaurant({"cuisine":"indian", "location":"Vizag"}))



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for indian food at Vizag. suggest me a fancy name for it.


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 21, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
suggest me some menu items for  I.

> Finished chain.

> Finished chain.
{'cuisine': 'indian', 'location': 'Vizag', 'restaurant_name': ' I', 'menu_items': 'e. I want to show the menu items in'}


In [ ]:
from langchain.llms import HuggingFaceHub

llm = HuggingFaceHub(repo_id="bigscience/bloom-1b7")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [ ]:
prompt = PromptTemplate(
    input_variables=["city"],
    template="Describe a perfect day in {city}?",
)
llmchain = LLMChain(llm=llm, prompt=prompt)

llmchain.run("Paris")


' A day in Paris is a day that you will never forget'

In [ ]:
name_prompt = PromptTemplate(
    input_variables=["cuisine", "location"],
    template="I want to open a restaurant for {cuisine} food at {location}. suggest me a fancy name for it."
)

menu_prompt = PromptTemplate(
    input_variables=['restaurant_name'],
    template = "suggest me some menu items for {restaurant_name}."
)

name_chain = LLMChain(llm = llm, prompt = name_prompt, verbose=True, output_key="restaurant_name")
menu_chain = LLMChain(llm = llm, prompt = menu_prompt, verbose = True, output_key="menu_items")

my_restaurant = SequentialChain(chains=[name_chain, menu_chain],
                                input_variables=['cuisine', 'location'],
                                output_variables=['restaurant_name', 'menu_items'],
                                verbose=True)


print(my_restaurant({"cuisine":"indian", "location":"Vizag"}))



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for indian food at Vizag. suggest me a fancy name for it.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
suggest me some menu items for  I.

> Finished chain.

> Finished chain.
{'cuisine': 'indian', 'location': 'Vizag', 'restaurant_name': ' I', 'menu_items': 'e. I want to show the menu items in'}
